In [1]:
CUDA_INDEX = 5
NAME = 'GED_LINUX'
CLASSES = 1

In [2]:
import sys
sys.path.insert(0, '../..')
sys.path.insert(0, '../../pyged/lib')

In [3]:
import os
import pickle
import random
import time

import IPython as ipy
import matplotlib.pyplot as plt
import numpy as np
import torch
torch.cuda.set_device(CUDA_INDEX)
torch.backends.cudnn.benchmark = True
import torch.optim
import torch_geometric as tg
import torch_geometric.data
from tqdm.auto import tqdm

from neuro import config, datasets, metrics, models, train, utils, viz
import pyged

from importlib import reload
reload(config)
reload(datasets)
reload(metrics)
reload(models)
reload(pyged)
reload(train)
reload(utils)
reload(viz)

<module 'neuro.viz' from '../../neuro/viz.py'>

In [ ]:
train_set, train_meta = torch.load(f'../data/{NAME}/train.pt', map_location='cpu')

In [ ]:
nodes = [h.num_nodes for h in train_set[1]]
edges = [h.num_edges for h in train_set[1]]
print(f'avg target nodes: {sum(nodes)/len(nodes):.3f}')
print(f'avg target edges: {sum(edges)/len(edges):.3f}')

In [ ]:
viz.plot_inner_dataset(train_set, n_items=5, random=True)

In [ ]:
viz.plot_dataset_dist(train_set)

In [ ]:
queries, targets, lb, ub = train_set
sed = (lb+ub)/2
plt.figure(figsize=(12,4))
plt.subplot(121)
viz.plot_corr(sed, ub-lb, show_std=True)
plt.xlabel('GED')
plt.ylabel('UB-LB')
plt.title('Range vs GED')
plt.subplot(122)
viz.plot_corr(sed, (ub-lb)/sed, show_std=True)
plt.xlabel('GED')
plt.ylabel('(UB-LB)/GED')
plt.title('Relative Range vs GED')
plt.show()

In [ ]:
val_set, _ = torch.load(f'../data/{NAME}/val.pt', map_location='cpu')

In [ ]:
model = models.NormGEDModel(8, CLASSES, 64, 64).to(config.device)
model.weighted = True

In [ ]:
loader = tg.data.DataLoader(list(zip(*train_set)), batch_size=200, shuffle=True)
val_loader = tg.data.DataLoader(list(zip(*val_set)), batch_size=1000, shuffle=True)

In [ ]:
dump_path = os.path.join(f'../runlogs/{NAME}-Norm', str(time.time()))
os.mkdir(dump_path)
train.train_full(model, loader, val_loader, lr=1e-3, weight_decay=1e-3, cycle_patience=5, step_size_up=2000, step_size_down=2000, dump_path=dump_path)